In [13]:
import boto3
import gzip
from io import BytesIO
from io import StringIO
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm.notebook import trange, tqdm
s3 = boto3.resource('s3')

In [7]:
BUCKET_NAME='cryptocompare-ohlcv-hourly'
my_bucket = s3.Bucket(BUCKET_NAME)

In [8]:
# Get file from s3 as binary, unzip, and read contents
def read_s3_file(bucketname, filename):
    obj = s3.Object(bucketname, filename)
    body = obj.get()['Body'].read()
    gzipfile = BytesIO(body)
    gzipfile = gzip.GzipFile(fileobj=gzipfile)
    content = gzipfile.read()
    s = str(content, 'utf-8')
    data = StringIO(s)
    df = pd.read_csv(data)
    df['timestamp']= pd.to_datetime(df['timestamp']) 
    df['symbol'] = filename.split('-')[0] + "/" + filename.split('-')[1]
    cols = df.columns.tolist()
    cols = cols[:-1]
    cols = ['symbol'] + cols
    df = df[cols]
    return df
    

In [9]:
def get_feature_df(df):
    f_df = df
    f_df['Date'] = f_df['timestamp']
    f_df['Open'] = f_df['open']
    f_df['High'] = f_df['high']
    f_df['Low'] = f_df['low']
    f_df['Close'] = f_df['close']
    f_df['High - Low'] = (f_df['high'] - f_df['low'])
    f_df['Number of Trades'] = ((f_df['volumefrom'] + f_df['volumeto']))
    f_df['Returns 1'] = f_df['close'].pct_change()
    f_df['Returns 2'] = f_df['close'].pct_change(periods=2)
    f_df['Returns 3'] = f_df['close'].pct_change(periods=3)
    f_df['Returns 4'] = f_df['close'].pct_change(periods=4)
    f_df['Returns 5'] = f_df['close'].pct_change(periods=5)
    f_df['MA5'] = f_df['close'].rolling(window=5).mean()
    f_df = f_df.drop(['timestamp', 'time', 'close', 'high', 'low', 'open', 'volumefrom',
       'volumeto'], axis=1)
    return f_df

In [10]:
df = read_s3_file(BUCKET_NAME, '1ST-BTC-BitTrex-hr.csv.gz')
features = get_feature_df(df)

In [11]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6003 entries, 0 to 6002
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   symbol            6003 non-null   object        
 1   Date              6003 non-null   datetime64[ns]
 2   Open              6003 non-null   float64       
 3   High              6003 non-null   float64       
 4   Low               6003 non-null   float64       
 5   Close             6003 non-null   float64       
 6   High - Low        6003 non-null   float64       
 7   Number of Trades  6003 non-null   float64       
 8   Returns 1         6002 non-null   float64       
 9   Returns 2         6001 non-null   float64       
 10  Returns 3         6000 non-null   float64       
 11  Returns 4         5999 non-null   float64       
 12  Returns 5         5998 non-null   float64       
 13  MA5               5999 non-null   float64       
dtypes: datetime64[ns](1), fl

In [12]:
files_list = []
for my_bucket_object in my_bucket.objects.all():
    # Download file and do something with data...
    file_name = my_bucket_object.key
    if file_name.endswith('.csv.gz'):
        if 'Binance' in file_name and '-BTC' in file_name:
#             file_info_arr = file_name.split('-')
#             base_currency = file_info_arr[0]
#             quote_currency = file_info_arr[1]
#             exchange = file_info_arr[2]
            files_list.append(file_name)

In [14]:
def getAllData(start_date):
    frames=[]
    for filename in tqdm(files_list, desc='OHLCV Files'):
        df = read_s3_file(BUCKET_NAME, filename)
        features_df = get_feature_df(df)
        dt_fmt = '%Y-%m-%d %H:%M:%S'
        start_dt = datetime.strptime(start_date, dt_fmt)
        frames.append(features_df[features_df['Date'] >= start_dt])
    return pd.concat(frames)
    
    
    
    

In [15]:
start_date = '2020-05-13 00:00:00'
df = getAllData(start_date)
# end_dt = datetime.strptime('2020-06-13 00:00:00', dt_fmt)

In [16]:
df

,symbol,Date,Open,High,Low,Close,High - Low,Number of Trades,Returns 1,Returns 2,Returns 3,Returns 4,Returns 5,MA5
23247,ADA/BTC,2020-05-13 00:00:00,0.000019,0.000019,0.000019,0.000019,4.400000e-07,1264619.280,0.006789,0.000519,0.007841,0.011542,0.042162,0.000019
23248,ADA/BTC,2020-05-13 01:00:00,0.000019,0.000019,0.000019,0.000019,4.900000e-07,1256910.230,-0.008299,-0.001567,-0.007784,-0.000523,0.003148,0.000019
23249,ADA/BTC,2020-05-13 02:00:00,0.000019,0.000019,0.000019,0.000019,4.700000e-07,923100.480,-0.009414,-0.017635,-0.010966,-0.017125,-0.009932,0.000019
23250,ADA/BTC,2020-05-13 03:00:00,0.000019,0.000019,0.000019,0.000019,4.300000e-07,1169372.270,0.016895,0.007322,-0.001037,0.005744,-0.000519,0.000019
23251,ADA/BTC,2020-05-13 04:00:00,0.000019,0.000019,0.000019,0.000019,4.900000e-07,816577.560,-0.007788,0.008976,-0.000523,-0.008817,-0.002089,0.000019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26008,ZRX/BTC,2020-06-13 16:00:00,0.000020,0.000020,0.000020,0.000020,1.700000e-07,775790.680,-0.002964,-0.002964,-0.000990,-0.001484,-0.006890,0.000020
26009,ZRX/BTC,2020-06-13 17:00:00,0.000020,0.000020,0.000020,0.000020,2.400000e-07,825310.650,-0.002478,-0.005435,-0.005435,-0.003465,-0.003958,0.000020
26010,ZRX/BTC,2020-06-13 18:00:00,0.000020,0.000020,0.000020,0.000020,1.700000e-07,499622.060,0.003974,0.001487,-0.001482,-0.001482,0.000495,0.000020
26011,ZRX/BTC,2020-06-13 19:00:00,0.000020,0.000020,0.000020,0.000020,1.100000e-07,172410.472,-0.003958,0.000000,-0.002478,-0.005435,-0.005435,0.000020


In [23]:
df = df.rename(columns={"symbol": "Symbol"})

In [24]:
df

,Symbol,Date,Open,High,Low,Close,High - Low,Number of Trades,Returns 1,Returns 2,Returns 3,Returns 4,Returns 5,MA5
23247,ADA/BTC,2020-05-13 00:00:00,0.000019,0.000019,0.000019,0.000019,4.400000e-07,1264619.280,0.006789,0.000519,0.007841,0.011542,0.042162,0.000019
23248,ADA/BTC,2020-05-13 01:00:00,0.000019,0.000019,0.000019,0.000019,4.900000e-07,1256910.230,-0.008299,-0.001567,-0.007784,-0.000523,0.003148,0.000019
23249,ADA/BTC,2020-05-13 02:00:00,0.000019,0.000019,0.000019,0.000019,4.700000e-07,923100.480,-0.009414,-0.017635,-0.010966,-0.017125,-0.009932,0.000019
23250,ADA/BTC,2020-05-13 03:00:00,0.000019,0.000019,0.000019,0.000019,4.300000e-07,1169372.270,0.016895,0.007322,-0.001037,0.005744,-0.000519,0.000019
23251,ADA/BTC,2020-05-13 04:00:00,0.000019,0.000019,0.000019,0.000019,4.900000e-07,816577.560,-0.007788,0.008976,-0.000523,-0.008817,-0.002089,0.000019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26008,ZRX/BTC,2020-06-13 16:00:00,0.000020,0.000020,0.000020,0.000020,1.700000e-07,775790.680,-0.002964,-0.002964,-0.000990,-0.001484,-0.006890,0.000020
26009,ZRX/BTC,2020-06-13 17:00:00,0.000020,0.000020,0.000020,0.000020,2.400000e-07,825310.650,-0.002478,-0.005435,-0.005435,-0.003465,-0.003958,0.000020
26010,ZRX/BTC,2020-06-13 18:00:00,0.000020,0.000020,0.000020,0.000020,1.700000e-07,499622.060,0.003974,0.001487,-0.001482,-0.001482,0.000495,0.000020
26011,ZRX/BTC,2020-06-13 19:00:00,0.000020,0.000020,0.000020,0.000020,1.100000e-07,172410.472,-0.003958,0.000000,-0.002478,-0.005435,-0.005435,0.000020


In [27]:
df['Symbol'] = df['Symbol'].str.replace('/', '')

In [28]:
df

,Symbol,Date,Open,High,Low,Close,High - Low,Number of Trades,Returns 1,Returns 2,Returns 3,Returns 4,Returns 5,MA5
23247,ADABTC,2020-05-13 00:00:00,0.000019,0.000019,0.000019,0.000019,4.400000e-07,1264619.280,0.006789,0.000519,0.007841,0.011542,0.042162,0.000019
23248,ADABTC,2020-05-13 01:00:00,0.000019,0.000019,0.000019,0.000019,4.900000e-07,1256910.230,-0.008299,-0.001567,-0.007784,-0.000523,0.003148,0.000019
23249,ADABTC,2020-05-13 02:00:00,0.000019,0.000019,0.000019,0.000019,4.700000e-07,923100.480,-0.009414,-0.017635,-0.010966,-0.017125,-0.009932,0.000019
23250,ADABTC,2020-05-13 03:00:00,0.000019,0.000019,0.000019,0.000019,4.300000e-07,1169372.270,0.016895,0.007322,-0.001037,0.005744,-0.000519,0.000019
23251,ADABTC,2020-05-13 04:00:00,0.000019,0.000019,0.000019,0.000019,4.900000e-07,816577.560,-0.007788,0.008976,-0.000523,-0.008817,-0.002089,0.000019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26008,ZRXBTC,2020-06-13 16:00:00,0.000020,0.000020,0.000020,0.000020,1.700000e-07,775790.680,-0.002964,-0.002964,-0.000990,-0.001484,-0.006890,0.000020
26009,ZRXBTC,2020-06-13 17:00:00,0.000020,0.000020,0.000020,0.000020,2.400000e-07,825310.650,-0.002478,-0.005435,-0.005435,-0.003465,-0.003958,0.000020
26010,ZRXBTC,2020-06-13 18:00:00,0.000020,0.000020,0.000020,0.000020,1.700000e-07,499622.060,0.003974,0.001487,-0.001482,-0.001482,0.000495,0.000020
26011,ZRXBTC,2020-06-13 19:00:00,0.000020,0.000020,0.000020,0.000020,1.100000e-07,172410.472,-0.003958,0.000000,-0.002478,-0.005435,-0.005435,0.000020


In [30]:
df.to_excel(r'pareto-multi-series-data2.xlsx')